# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv
%reload_ext dotenv


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

# Load the environment variable FEATURES_DATA
FEATURES_DATA = os.getenv("FEATURES_DATA")

if FEATURES_DATA:
    # Use glob to find all Parquet files in the FEATURES_DATA directory
    parquet_files = glob(os.path.join(FEATURES_DATA, "*.parquet"))
    
    # Print the list of found Parquet files
    print("Parquet files found in FEATURES_DATA:")
    for file in parquet_files:
        print(file)
else:
    print("Environment variable FEATURES_DATA is not set.")



Parquet files found in FEATURES_DATA:
../../05_src/data/features/stock_features.parquet\part.0.parquet
../../05_src/data/features/stock_features.parquet\part.1.parquet
../../05_src/data/features/stock_features.parquet\part.10.parquet
../../05_src/data/features/stock_features.parquet\part.100.parquet
../../05_src/data/features/stock_features.parquet\part.101.parquet
../../05_src/data/features/stock_features.parquet\part.102.parquet
../../05_src/data/features/stock_features.parquet\part.103.parquet
../../05_src/data/features/stock_features.parquet\part.104.parquet
../../05_src/data/features/stock_features.parquet\part.105.parquet
../../05_src/data/features/stock_features.parquet\part.106.parquet
../../05_src/data/features/stock_features.parquet\part.107.parquet
../../05_src/data/features/stock_features.parquet\part.108.parquet
../../05_src/data/features/stock_features.parquet\part.109.parquet
../../05_src/data/features/stock_features.parquet\part.11.parquet
../../05_src/data/features/sto

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
import os
import dask.dataframe as dd
from glob import glob

# Load the environment variable FEATURES_DATA
FEATURES_DATA = os.getenv("FEATURES_DATA")

if FEATURES_DATA:
    # Use glob to find all Parquet files in the FEATURES_DATA directory
    parquet_files = glob(os.path.join(FEATURES_DATA, "*.parquet"))
    
    # Read Parquet files into a Dask DataFrame
    dd_px = dd.read_parquet(parquet_files)
    
    # Print the first few rows to inspect the columns
    print(dd_px.head())
    
    # Print the columns
    print("Columns in the DataFrame:", dd_px.columns)
else:
    print("Environment variable FEATURES_DATA is not set.")


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
OXY    2000-01-03  10.269221  10.388979   9.939887   9.969827   5.047811   
OXY    2000-01-04   9.790190  10.059645   9.730311   9.760251   4.941700   
OXY    2000-01-05   9.760251   9.909948   9.670432   9.820130   4.972018   
OXY    2000-01-06   9.730311  10.269221   9.700372  10.089584   5.108444   
OXY    2000-01-07  10.329100  10.778191  10.209342  10.388979   5.260029   

         Volume  sector                           subsector  year  \
ticker                                                              
OXY     1161303  Energy  Oil & Gas Exploration & Production  2000   
OXY     1576308  Energy  Oil & Gas Exploration & Production  2000   
OXY     1172159  Energy  Oil & Gas Exploration & Production  2000   
OXY     3336738  Energy  Oil & Gas Exploration & Production  2000   
OXY     3325674  Energy  Oil & Gas Exploration & Prod

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
import os
import dask.dataframe as dd
from glob import glob

# Load the environment variable FEATURES_DATA
FEATURES_DATA = os.getenv("FEATURES_DATA")

if FEATURES_DATA:
    # Use glob to find all Parquet files in the FEATURES_DATA directory
    parquet_files = glob(os.path.join(FEATURES_DATA, "*.parquet"))
    
    if parquet_files:
        # Read Parquet files into a Dask DataFrame
        dd_px = dd.read_parquet(parquet_files)
        
        # Convert Dask DataFrame to Pandas DataFrame
        df_px = dd_px.compute()
        
        # Check if 'returns' column exists and calculate the rolling average return with a window of 10 days
        if 'returns' in df_px.columns:
            df_px['rolling_avg_return'] = df_px['returns'].rolling(window=10).mean()
            # Print the first few rows to inspect the columns
            print(df_px.head())
        else:
            print("'returns' column not found in the DataFrame.")
        
        # Print the columns
        print("Columns in the DataFrame:", df_px.columns)
    else:
        print("No Parquet files found in the directory.")
else:
    print("Environment variable FEATURES_DATA is not set.")


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
OXY    2000-01-03  10.269221  10.388979   9.939887   9.969827   5.047811   
OXY    2000-01-04   9.790190  10.059645   9.730311   9.760251   4.941700   
OXY    2000-01-05   9.760251   9.909948   9.670432   9.820130   4.972018   
OXY    2000-01-06   9.730311  10.269221   9.700372  10.089584   5.108444   
OXY    2000-01-07  10.329100  10.778191  10.209342  10.388979   5.260029   

         Volume  sector                           subsector  year  \
ticker                                                              
OXY     1161303  Energy  Oil & Gas Exploration & Production  2000   
OXY     1576308  Energy  Oil & Gas Exploration & Production  2000   
OXY     1172159  Energy  Oil & Gas Exploration & Production  2000   
OXY     3336738  Energy  Oil & Gas Exploration & Production  2000   
OXY     3325674  Energy  Oil & Gas Exploration & Prod

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    No, it's not strictly necessary to convert to a Pandas DataFrame to calculate the moving average return. Dask DataFrames support similar operations, including rolling calculations. This approach takes advantage of Dask's parallel computing capabilities, which is especially useful for large datasets.
+ Would it have been better to do it in Dask? Why?
    Yes, it would generally be better to perform the rolling average calculation directly in Dask, especially if you are working with a large dataset. Here are some of the reason for using Dask
        Parallel Computing: Dask uses multiple cores or a distributed computing environment to speed up calculations.
        Out-of-Core Computation: Dask processes data in chunks, allowing it to handle datasets larger than the memory capacity of a single machine.
        Efficiency: Staying within Dask avoids the memory-intensive operation of converting to a Pandas DataFrame, which can be problematic for large datasets.
        Scalability: Dask scales easily from single machines to large clusters, accommodating growing data and computational needs.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.